### Import tools

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import pickle as pkl
import csv

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

### Load the data

In [ ]:
df_labels= pd.read_csv("../input/traffic-signs-preprocessed/label_names.csv")
train_imgs= pkl.load(open("../input/traffic-signs-preprocessed/train.pickle", "rb"))
test_imgs= pkl.load(open("../input/traffic-signs-preprocessed/test.pickle", "rb"))
valid_imgs= pkl.load(open("../input/traffic-signs-preprocessed/valid.pickle", "rb"))

labels= pkl.load(open("../input/traffic-signs-preprocessed/labels.pickle", "rb"))

In [ ]:
# Printing the labels
dict={}
reader= csv.reader(labels)

i=0

for row in reader:
    i+=1
    dict[i]= row[0]

print(dict)

### Split the images

In [ ]:
X_train= train_imgs["features"]
y_train= train_imgs["labels"]

X_val= valid_imgs["features"]
y_val= valid_imgs["labels"]

X_test= test_imgs["features"]
y_test= test_imgs["labels"]

In [ ]:
def shapes(X, y):

    print("X: ", X.shape)
    print("y: ", y.shape)
    

shapes(X_train, y_train)
shapes(X_val, y_val)
shapes(X_test, y_test)

### Visualize the data

In [ ]:
fig, ax= plt.subplots(3)

ax[0].imshow(X_train[0])
ax[1].imshow(X_val[10])
ax[2].imshow(X_test[3])

### Model Evaluation

In [ ]:
model= Sequential([
    
    Conv2D(32, kernel_size=(3, 3), input_shape=(32, 32, 3),
           activation= "relu"),

    MaxPooling2D(2, 2),
    
    Dropout(0.2),
    
    Conv2D(64, kernel_size=(3, 3),
           activation="relu"),
    
    MaxPooling2D(2, 2),
    
    Dropout(0.2),
    
    Flatten(),
    
    Dense(256, activation="relu"),
    
    Dropout(0.2),
    
    Dense(128, activation="relu"),
    
    Dropout(0.2),
    
    Dense(64, activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
reduceLr= ReduceLROnPlateau(monitor= "val_loss", 
                            factor= 0.3, mode= min, 
                            verbose=1, min_lr= 1e-5)

checkpoint= ModelCheckpoint("trafficModel.h5", save_best_only= True,
                            monitor= "val_loss", mode="min")

earlyStopping= EarlyStopping(monitor= "val_loss", restore_best_weights= True,
                             patience= 5, mode= "min")


In [ ]:
hist= model.fit(X_train, y_train, epochs=20,
                validation_data=(X_test, y_test), batch_size=32,
                callbacks=[earlyStopping, checkpoint, reduceLr])


### Plot the `accuracy` of the model

In [ ]:
plt.plot(hist.history["accuracy"], label="train")
plt.plot(hist.history["val_accuracy"], label= "validation")
plt.xlabel("Acc")
plt.ylabel("epochs")

plt.legend(["train", "test"])